In [14]:
import torch
from torch import nn
from torchvision import transforms
from torchvision import models
from PIL import Image
from glob import glob

In [15]:
class Net(nn.Module):
    def __init__(self, *args, **kwargs) -> None:
        super().__init__(*args, **kwargs)
        self.base_model = models.efficientnet_b0(
            weights=models.EfficientNet_B0_Weights.DEFAULT
        )
        self.layer = nn.Sequential(nn.Flatten(), nn.Linear(1000, 2))

    def forward(self, x):
        x = self.base_model(x)
        return self.layer(x)

In [33]:
device = torch.device("mps" if torch.backends.mps.is_available() else "cpu")

model_data = torch.load(
    "PneumoniaClassificationModel.pth", map_location=device
)

classifier = model_data["model"]
classifier.load_state_dict(model_data["state_dict"])

# device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
classifier.to(device)
classifier = classifier.module

print("Model Loaded on", device)

Model Loaded on mps


In [35]:
# Test
classes = ["NORMAL", "PNEUMONIA"]

transformer = transforms.Compose(
    [
        transforms.Resize((256, 256)),
        transforms.ToTensor(),
        transforms.Normalize((0.485, 0.456, 0.406), (0.229, 0.224, 0.225)),
    ]
)

images = glob("Tests/*.jpeg")

for image in images:
    image = Image.open(image).convert("RGB")
    with torch.no_grad():
        image_tensor = transformer(image)
        image_tensor = torch.unsqueeze(image_tensor, 0)
        image_tensor = image_tensor.to(device)

        output = classifier(image_tensor)
        prediction = torch.argmax(output.data, 1)
        print(classes[prediction.item()])

PNEUMONIA
PNEUMONIA
PNEUMONIA
PNEUMONIA
